In [1]:
#!pip install geopandas
#!pip install rasterstats
#!pip install descartes

**Dependencies**

In [2]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import zipfile
from pathlib import Path
import re
#import rasterstats # For zonal statistics. Extracts statistics from rasters files or numpy arrays based on geometries.
#import scikitlearn #The best and at the same time easy-to-use Python machine learning library. Regression, classification, dimensionality reductions etc.
import folium 
#import PySAL #The Python Spatial Analysis Library contains a multitude of functions for spatial analysis, statistical modeling and plotting.

%matplotlib inline

In [3]:
pd.options.display.max_rows
pd.options.display.max_rows = 999
pd.options.display.max_columns= 100

# Data

In [4]:
# READ Data 
FAMEWS = pd.read_csv("Data/FAMEWS_12February2019.csv", sep=',' , encoding='latin-1')
alpha_2= pd.read_csv("Data/Alpha_2.csv",sep=',' , encoding='latin-1')
alpha_2

# Join with country names
FAMEWS= FAMEWS.merge(alpha_2,left_on="country",right_on= "Code",how="left")

FileNotFoundError: [Errno 2] File b'Data/FAMEWS_12February2019.csv' does not exist: b'Data/FAMEWS_12February2019.csv'

In [ ]:
FAMEWS

In [ ]:
FAMEWS.columns
#rapsConfirmedFAW, FAWcropdamage

In [ ]:
# List of columns 
FAMEWS_new = FAMEWS[['_id','owner','deleted', 'date',
       'scouting', 'traps', 'country', 'Name','region', 'locationName',
       'trainingReceived', 'cropMain', 'cropVariety',
       'panelColumns5Maincropplantingdate', 'cropIrrigation', 'cropFertilizer',
       'cropStage', 'cropHealth', 'cropSystem', 'cropFieldSizeUnit',
       'rainLastDate', 'latitude', 'longitude', 'rotationIntercroppingCrop',
       'cropFieldSize', 'rainAmount', 'trapID', 'trapLocation', 'trapType',
       'trapCondition', 'trapLureName', 'trapReplaced', 'trapReplacedDate',
       'trapLureReplaced', 'trapKLureReplacedDate', 'btnGpsPosition',
       'checked', 'faw', 'sample1PlantsChecked', 'sample2PlantsChecked',
       'sample3PlantsChecked', 'sample4PlantsChecked', 'sample5PlantsChecked',
       'totalPlantsChecked', 'pestStageFAW', 'fawColumns4Cobdamage',
       'fawNaturalEnemies', 'fawLarvaeKilledByNaturalEnemies',
       'fawControlUndertaken', 'scoutingPlantsFAW', 'scoutingPercentageFAW',
       'totalFAW', 'totalAAW', 'totalBorer', 'scoutingStageFAW', 'userCountry',
        'fawCurrentDamage', 'fawPreviousDamage',
       'fawControlChemicalPesticideName', 'fawControlChemicalPesticideLitres',
       'fawControlLocalTypes', 'fawControlBiopesticideName',
       'fawControlBiopesticideLitres', 'sample1FAW', 'sample2FAW',
       'sample3FAW', 'sample4FAW', 'sample5FAW', 'access', 'form',
       'externalIds', 'created', 'modified','Code']]

In [ ]:
# insepecting first few rows 
FAMEWS_new.head()

---
# Data Cleaning 
---

## Temporality 
---

* Checking for basic dependencies: Program was launched on 2018 (2018-03-03T17:13:26.000Z). Check how many rows appear before this date and remove them.

In [ ]:
FAMEWS_new[FAMEWS_new['date'] < '2018-03-03T17:13:26.000Z'].shape[0]

114 entry points 
* Notice that 9 rows are from `1970-01-01T23:24:04.000Z` error - data was misread
* Notice that 4 rows are from  `2011-11-03T09:41:00.000Z`
* Notice that 101 rows are from `2018-01-01T00:51:00.000Z`, `2018-02-27T13:43:00.000Z`, `2018-03-01T14:05:55.000Z` - Trials before release?

**Remove 144 entries**

In [ ]:
FAMEWS_temp = FAMEWS_new[FAMEWS_new['date'] >= '2018-03-03T17:13:26.000Z']

---
## Duplicates 
---

In [ ]:
# number of rows 
len(FAMEWS_temp)

**Drop duplicates**

In [ ]:
FAMEWS_dup= FAMEWS_temp.drop_duplicates()
len(FAMEWS_dup)

#removed 10871 rows 

---
## Select Columns
---

## 1.
**Want to know if it is worth keeping totalFAW, totalAAW, and totalBorer or it was just accounted for in scoutingPlantsFAW**

In [ ]:
FAMEWS_dup[FAMEWS_dup["totalFAW"]>=0.0].shape
# (21050, 73)- Helps remove NaN values for this case. Also- helped confrimed that there are values on this column. 
# Worth keeping for now. 

In [ ]:
selected = ["_id",'owner', "date",
            "Name","country",
            "region","locationName","latitude", "longitude",
            "scouting", "traps","trainingReceived",
            "cropMain", "cropVariety", "cropIrrigation", 
            "cropFertilizer","cropStage","cropStage",
            "cropHealth", "cropSystem", "cropFieldSizeUnit", 
            "cropFieldSize", "rotationIntercroppingCrop",
            "rainLastDate","rainAmount", 
            
            "checked", "faw", 
            
            "scoutingPlantsFAW", "scoutingPercentageFAW", 
            "totalFAW", "totalAAW", "totalBorer",
           
            "trapType","trapCondition"]

FAMEWS_selected= FAMEWS_dup[selected]
FAMEWS_selected.head(5)

---
## Transformations

* column edits
---

## 1. 
**cropFieldSizeUnit [acre/ha] & cropFieldSize [numeric]** - Chose standard field sieze unit. Useful when checking spread/region 

In [ ]:
FAMEWS_selected.groupby(["cropFieldSizeUnit"]).count()["_id"]

In [ ]:
# # from acre to ha - divide acre by 2.471
# # from m2 to ha - 	 divide the area value by 10000
# # from yd2 to ha - for an approximate result, divide the area value by 11959.9

by_acre= FAMEWS_selected[FAMEWS_selected["cropFieldSizeUnit"] == "acre"]
by_acre.insert(21, "cropFieldSize_ha", by_acre["cropFieldSize"]/2.471, True) 
by_acre.insert(21,"cropFieldSizeUnit_ha", len(by_acre["cropFieldSize"])*["ha"])

by_m2= FAMEWS_selected[FAMEWS_selected["cropFieldSizeUnit"] == "m2"]
by_m2.insert(21, "cropFieldSize_ha", by_m2["cropFieldSize"]/1000, True) 
by_m2.insert(21,"cropFieldSizeUnit_ha", len(by_m2["cropFieldSize"])*["ha"])

by_yd2= FAMEWS_selected[FAMEWS_selected["cropFieldSizeUnit"] == "yd2"]
by_yd2.insert(21, "cropFieldSize_ha", by_yd2["cropFieldSize"]/11959.9, True) 
by_yd2.insert(21,"cropFieldSizeUnit_ha", len(by_yd2["cropFieldSize"])*["ha"])


by_ha= FAMEWS_selected[FAMEWS_selected["cropFieldSizeUnit"] == "ha"]
by_ha.insert(21, "cropFieldSize_ha", by_ha["cropFieldSize"], True) 
by_ha.insert(21,"cropFieldSizeUnit_ha", len(by_ha["cropFieldSize"])*["ha"])


all_dfs = [by_acre, by_m2, by_yd2,by_ha]
FAMEWS_selected= pd.concat(all_dfs).reset_index(drop=True)

FAMEWS_selected.drop(["cropFieldSizeUnit","cropFieldSize"],axis=1,inplace=True)
FAMEWS_selected.head()

### 2. 

**Replacing NaN values in**

* totalFAW	
* totalAAW	
* totalBorer
* checked
* faw	
* scoutingPlantsFAW	
* scoutingPercentageFAW	

In [ ]:
FAMEWS_selected.faw[1]

---
## New Features 
---

## 1. 

**days_since_last_rain**

**New column** number of days since last rain. Use **rainLastDate**- date-time and get difference from the date in which it was reported **date**

In [ ]:
from datetime import datetime
from datetime import date
from datetime import time


In [ ]:
#convert date to datetime 
FAMEWS_selected['rainLastDate']=pd.to_datetime(FAMEWS_selected['rainLastDate'])
#convert rainLastDate to datetime
FAMEWS_selected['date']=pd.to_datetime(FAMEWS_selected['date'])

In [ ]:
# Extract just days variables from date
FAMEWS_selected.insert(2,"date_date", FAMEWS_selected["date"].apply(lambda x: x.date()), True)  # comment out true to avoid getting a another column 

# Extract just days variables from rainLastDate_date
FAMEWS_selected.insert(25,"rainLastDate_date", FAMEWS_selected["rainLastDate"].apply(lambda x: x.date()), True) 

In [ ]:
FAMEWS_selected.insert(26,"rainLastDate_date", FAMEWS_selected["date_date"]-FAMEWS_selected["rainLastDate_date"],True)

In [ ]:
FAMEWS_selected.head(10)

## 2. 
**Days since first reporting**
* Need to confirm the date of the first reporting for each country 
* Alternatevely, can add use the first daye reported using FAMWES as the initial date- but this could be very misleading. 

---
## EDA
---

### Country

In [ ]:
countries = FAMEWS_selected.groupby(["Name"]).count().sort_values("_id", ascending=False)['_id'].to_frame()

In [ ]:
countries = countries.reset_index()


In [ ]:
print(f' We have survey data for {len(countries)} countries')

In [ ]:
plt.figure(figsize=(20,10 ))
ax=sns.barplot(x="_id", y="Name", data=countries)
ax.set(xlabel='Survey Count', ylabel='Country Name')
plt.show()

### Training Received

In [ ]:
training=FAMEWS_selected.groupby(["trainingReceived"]).count().sort_values("_id", ascending=False)['_id'].to_frame()

In [ ]:
training = training.reset_index()
training

In [ ]:
print(f' 1. Main training received are from {training["trainingReceived"][0]}, {training["_id"][0]} counts.')
print(f' 2. Main training received are from {training["trainingReceived"][1]}, {training["_id"][1]} counts.')
print(f' 3. Main training received are from {training["trainingReceived"][2]}, {training["_id"][2]} counts.')
print(f' 4. Main training received are from {training["trainingReceived"][3]}, {training["_id"][3]} counts.')
print(f' 5. Main training received are from {training["trainingReceived"][4]}, {training["_id"][4]} counts.')

In [ ]:
FAMEWS_selected.head(3)

### Crops

In [ ]:
by_crop=FAMEWS_selected.groupby("cropMain").count().sort_values("_id",ascending=False)['_id'].to_frame().reset_index()

print(f' We have {len(by_crop["cropMain"])} crops')
print()
print(f' The crops that we have are {by_crop["cropMain"].sort_values()}')
print('Notice the "Select". This means that some farmers did not select a crop')

**Main crop count (All countries)**

In [ ]:
plt.figure(figsize=(20,10 ))
ax=sns.barplot(x="_id", y="cropMain", data=by_crop)
ax.set(xlabel='count', ylabel='Main Crop')
plt.show()

**Number of crops per country**

In [ ]:
main_crop1=FAMEWS_selected.groupby(["Name","cropMain"]).count().sort_values("_id", ascending=False)['_id'].to_frame()
main_crop1= main_crop1.reset_index()
main_crop1=main_crop1.sort_values(["Name",'_id'], ascending=[True, False])
main_crop1.head(4)


**Crop per region in each country**

In [ ]:
main_crop2=FAMEWS_selected.groupby(["Name","region","cropMain"]).count().sort_values("_id", ascending=False)['_id'].to_frame()
main_crop2= main_crop1.reset_index()
main_crop2.sort_values(['Name', '_id'], ascending=[False, False])

**Main Crops By Country**

In [ ]:
by_crop.sort_values("_id",ascending=False)

In [ ]:
crops = dict()
for crop in by_crop.cropMain:
    crops[crop] = main_crop1[main_crop1['cropMain'] == crop].sort_values("_id", ascending=False)

In [ ]:
for key in crops.keys():    
    plt.figure(figsize=(20,10 ))
    ax=sns.barplot(x="_id", y="Name", data=crops[key])
    ax.set(xlabel='Survey Count', ylabel='Country Name', title=key.upper())
    plt.show()

In [ ]:
#trapsConfirmedFAW, FAWcropdamage

## 1. 

### FAW by main crop

## 2.

### FAW by crop stage 

## 3.

### FAW by crop system 

## Outliers 





## Buffer Zones

In [ ]:
# Check CRS -- World Mercator, units of meters.
from shapely.geometry import Point, Polygon
#FAMEWS_new['geometry']= [Point(x,y).buffer(500000) for x,y in zip(FAMEWS_new['longitude'],FAMEWS_new['latitude'])]
#FAMEWS_new['geometry']= FAMEWS_new['geometry'].apply(Point)
#crs ={'init':'epsg:3395'}
geo_FAMEWS= gpd.GeoDataFrame([
    {'geometry': Point(x,y).buffer(500000),"latitude":y, "longitude":x}
    for x,y in zip(FAMEWS_new['longitude'],FAMEWS_new['latitude'])])
# Previously inside of the Geo Data Frame - FAMEWS_new[["Name", "date","longitude","latitude"]],geometry="geometry")
print(geo_FAMEWS.head())
#geo_FAMEWS['geometry']=geo_FAMEWS.buffer(500000)
#geo_FAMEWS.plot()


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# Select Africa and some columns
region= world[(world['continent'] != "South America") &(world['continent'] != "North America")&(world['continent'] != "Antarctica")]
#region['geometry'].crs=({'init':'epsg:3395'})
ax=region.plot(figsize=(50, 50), alpha=0.5, edgecolor='k')
geo_FAMEWS.plot(ax=ax, color='green')

In [ ]:
#make 500km buffer
#geo_FAMEWS['geometry']=geo_FAMEWS.buffer(500000)
#holes=geo_FAMEWS.buffer(500000)
geo_FAMEWS.plot(markersize=20,facecolors='none',edgecolor='r',alpha=0.5)

Possibly locations to look into (coordiates)

In [ ]:
#region.head()
#geo_FAMEWS=geo_FAMEWS[["Name","date","longitude","latitude","geometry"]]
type(geo_FAMEWS)
#newdf = gpd.overlay(region, geo_FAMEWS, how="union")

In [ ]:
#lat top = 19, bottom = -35
#long left = -20, right = 52
#FAMEWS_new[(FAMEWS_new['latitude'] >=-35)  &  (FAMEWS_new['latitude'] <=19)& (FAMEWS_new['longitude'] <=52)& (FAMEWS_new['longitude'] >= -20)]


Number of regions

In [ ]:
len(FAMEWS_unique['region'].unique())

Number of counries

In [ ]:
len(FAMEWS_unique['country'].unique())

# Zambia

In [ ]:
Zambia_training=Zambia.groupby(['trainingReceived']).count()[["_id"]].reset_index()

#--- Total
total= sum(Zambia_training["_id"])

#---extensionService
extensionService= Zambia_training[Zambia_training['trainingReceived']=='extensionService']["_id"]
extensionService_prop=(extensionService/total)*100 
extensionService_prop

#--- FAO
fao= Zambia_training[Zambia_training['trainingReceived']=='fao']["_id"]
fao_prop=(fao/total)*100 

#--- None
no_training= Zambia_training[Zambia_training['trainingReceived']=='none']["_id"]
no_training_prop=(no_training/total)*100 

#---
print(extensionService_prop,fao_prop,no_training_prop)


In [ ]:
Zambia.groupby(['trainingReceived']).count()["_id"].plot.bar()

In [ ]:
len(Zambia.groupby(['region']).count())

# Ghana 

In [ ]:
Ghana= FAMEWS_unique[FAMEWS_unique['Name']=='Ghana']
Ghana.head(10)

In [ ]:
Ghana.groupby(['trainingReceived']).count()

In [ ]:
Ghana_training=Ghana.groupby(['trainingReceived']).count()[["_id"]].reset_index()

Ghana_training.sort_values("_id", ascending=False)

In [ ]:
#--- Total
total= sum(Ghana_training["_id"])

#---extensionService
extensionService_g= Ghana_training[Ghana_training['trainingReceived']=='extensionService']["_id"]
extensionService_g_prop=(extensionService_g/total)*100 
extensionService_g_prop

#--- FAO
fao_ext_g= Ghana_training[Ghana_training['trainingReceived']=='extensionService,fao']["_id"]
fao_ext_g_prop=(fao_ext_g/total)*100 



#---
print(extensionService_g_prop,fao_ext_g_prop)

In [ ]:
len(Ghana.groupby(['region']).count())

# Ethiopia 

In [ ]:
Ethiopia= FAMEWS_unique[FAMEWS_unique['Name']=='Ethiopia']
Ethiopia.head(10)

In [ ]:
Ethiopia_training=Ethiopia.groupby(['trainingReceived']).count()[["_id"]].reset_index()

Ethiopia_training.sort_values("_id", ascending=False)

In [ ]:

#--- Total
total_e= sum(Ethiopia_training["_id"])

#---extensionService
extensionService_e= Ethiopia_training[Ghana_training['trainingReceived']=='extensionService']["_id"]
extensionService_e_prop=(extensionService_e/total_e)*100 
extensionService_e_prop

#--- FocalPerson
focal_e= Ethiopia_training[Ethiopia_training['trainingReceived']=='focalPerson']["_id"]
focal_e_prop=(focal_e/total_e)*100 

#--- fao
fao_e= Ethiopia_training[Ethiopia_training['trainingReceived']=='fao']["_id"]
fao_e_prop=(fao_e/total_e)*100 

#--- extensionService,fao
#fao_ext_e= Ethiopia_training[(Ethiopia_training['trainingReceived']=='extensionService,fao')& (Ethiopia_training['trainingReceived']=='fao,extensionService')]["_id"]
fao_ext_e_prop=((641+246)/total_e)*100 


#---
print(extensionService_e_prop,focal_e_prop,fao_e_prop,fao_ext_e_prop)

In [ ]:
len(Ethiopia.groupby(['region']).count())

# Mozambique 

In [ ]:
len(gen1[gen1['country']=='MZ'])

In [ ]:
gen1[gen1['country']=='MZ'].head(10)

In [ ]:
MZ_unique= gen1[gen1['country']=='MZ'].drop_duplicates()
len(MZ_unique)
# unique entries - 1467

In [ ]:
# testing in how many variables does the variables differ
diff_count = (MZ_unique.iloc[1] == MZ_unique.iloc[0])
diff_count.value_counts()
# True     49
# False    27


In [ ]:
# Identifying how are each row distinct (e.g. same user id, but several entries)
#diff_count=diff_count.to_frame()
diff_count[diff_count[0]==False].head()

In [ ]:
MZ_unique_owner=MZ_unique['owner'].unique()
len(MZ_unique_owner)
# owners - 90 

In [ ]:
MZ_unique_user=MZ_unique['_id'].unique()
len(MZ_unique_user)
# unqiue ID's-  609 

In [ ]:
MZ_unique_form=MZ_unique['form'].unique()
len(MZ_unique_form)
# unique forms- where am I getting the 382 surveys 